## 10.4 Shoppy

W tej apllikacji zaimplementujemy architekturę `MVVM`. Aplikacja będzie listą zakupową z bazą `ROOM` umożliwiająca dodanie elementu, usunięcie, edycję oraz wyszukanie. Wykorzystamy `Jetpack Navigation`do nawigowania po trzech fragmentach
- `ListFragment` - fragment domowy wyświetlający listę wszystkich elementów oraz pozwalający przeszukać bazę
- `AddFragment` - przejście z `ListFragment` za pomocą `FAB` - umożliwia dodanie nowego elementu do bazy
- `UpdateFragment` - przejście przez kliknięcie elementu `RecyclerView` na `ListFragment` - umożliwia edycję wybranego elementu.

Dodajmy wymagane zależności do plików `build.gradle`

In [ ]:
// Project
buildscript {
    repositories {
        google()
    }
    dependencies {
        classpath "androidx.navigation:navigation-safe-args-gradle-plugin:2.5.1"
    }
}

In [ ]:
// Module
plugins {
    id 'com.android.application'
    id 'androidx.navigation.safeargs'
}
...
buildFeatures {
    viewBinding true
}
...
dependencies {

    // Navigation
    implementation "androidx.navigation:navigation-fragment:2.5.1"
    implementation "androidx.navigation:navigation-ui:2.5.1"

    // ROOM
    implementation "androidx.room:room-runtime:2.4.3"
    annotationProcessor "androidx.room:room-compiler:2.4.3"

    // ViewModel
    implementation 'androidx.lifecycle:lifecycle-viewmodel:2.5.1'

    // LiveData
    implementation "androidx.lifecycle:lifecycle-livedata:2.5.1"
    ...
}

### **data**

Rozpocznijmy od zdefiniowania modelu - klasa `Item` będzie posiadała trzy pola
- `id` - unikalny identyfikator
- `name` - nazwa produktu
- `quantity` - ilość danego produktu
Nasza klasa będzie reprezentowała tabelę w bazie `ROOM`, więc dodamy również adnotację `@Entity`

In [ ]:
@Entity(tableName = "item_table")
public class Item {

    @PrimaryKey(autoGenerate = true)
    private final int id;
    private final String name;
    private final int quantity;

    public Item(int id, String name, int quantity) {
        this.id = id;
        this.name = name;
        this.quantity = quantity;
    }

    public String getName() {
        return name;
    }

    public int getQuantity() {
        return quantity;
    }

    public int getId() {
        return id;
    }
}

Wykorzystujemy pole `id` jako `PrimaryKey` który będzie automatycznie generowany przez `ROOM`.

Zdefiniujmy nasz `DAO` z odpowiednimi metodami

In [ ]:
@Dao
public interface ItemDao {
    @Insert(onConflict = OnConflictStrategy.IGNORE)
    void addItem(Item item); // dodaj element

    @Query("SELECT * FROM item_table ORDER BY id ASC")
    LiveData<List<Item>> readAllData(); // czytaj wszystkie elementy

    @Query("SELECT * FROM item_table WHERE id = :id")
    LiveData<Item> getItem(int id); // czytaj element o zadanym id

    @Update(onConflict = OnConflictStrategy.REPLACE)
    void updateItem(Item item); // aktualizuj element

    @Delete
    void deleteItem(Item item); // usuń element

    @Query("DELETE FROM item_table")
    void deleteAll(); // usuń wszystkie elementy

    @Query("SELECT * FROM item_table WHERE name LIKE :query")
    LiveData<List<Item>> searchItem(String query); // wyszukaj element o zadanej nazwie
}

Kolejnym elementem będzie baza danych `ROOM` - tak jak poprzednio wykorzystamy `ExecutorService` to zapisu do bazy. Z bazy dane zwrócimy jako `LiveData`

In [ ]:
@Database(entities = {Item.class}, version = 1, exportSchema = false)
abstract public class ItemDatabase extends RoomDatabase {

    public abstract ItemDao itemDao();

    private static volatile ItemDatabase INSTANCE;

    private static final int NUMBER_OF_THREADS = 4;
    public static final ExecutorService databaseWriteExecutor =
            Executors.newFixedThreadPool(NUMBER_OF_THREADS);

    public static ItemDatabase getDatabase(final Context context) {
        if (INSTANCE == null) {
            synchronized (ItemDatabase.class) {
                if (INSTANCE == null) {
                    INSTANCE = Room.databaseBuilder(context.getApplicationContext(),
                                    ItemDatabase.class, "item_database_java")
                            .build();
                }
            }
        }
        return INSTANCE;
    }
}

Następnie zdefiniujmy `Repository` - element nie należący do architektury `MVVM` lecz mocno zalecany. Dodajmy dwa pola `ItemDao` oraz `readAllData`

In [ ]:
public class ItemRepository {

    private final ItemDao itemDao;
    private final LiveData<List<Item>> readAllData;

W konstruktorze inicjujemy `itemDao` oraz `readAllData`

In [ ]:
    public ItemRepository(Application application){
        ItemDatabase db = ItemDatabase.getDatabase(application);
        itemDao = db.itemDao();
        readAllData = itemDao.readAllData();
    }

I wystawiamy wszystkie metody interfejsu `ItemDao`

In [ ]:
    public LiveData<List<Item>> ReadAllData() {
        return readAllData;
    }

    public LiveData<Item> getItem(int id){
        return itemDao.getItem(id);
    }

    public void delete(Item item){
        itemDao.deleteItem(item);
    }

    public void update(Item item){
        itemDao.updateItem(item);
    }

    public void insert(Item item) { itemDao.addItem(item); }

    public void deleteAll(){ itemDao.deleteAll(); }

    public LiveData<List<Item>> searchItem(String query){
        return itemDao.searchItem(query);
    }

Ostatnim elementem danych będzie `ItemViewModel` - dodajmy dwa pola przechowujące `Repository` oraz listę wszystkich elementów 

In [ ]:
public class ItemViewModel extends AndroidViewModel {

    private final ItemRepository repository;

    private final LiveData<List<Item>> readAllData;

W konstruktorze zainicjujmy te dwa pola

In [ ]:
    public ItemViewModel(@NonNull Application application) {
        super(application);
        repository = new ItemRepository(application);
        readAllData = repository.ReadAllData();
    }

Dodajmy metody odczytujące dane z bazy i zwracające `LiveData`

In [ ]:
    public LiveData<List<Item>> getAllData() { // getter
        return readAllData;
    }

    public LiveData<Item> getItem(int id){
        return repository.getItem(id);
    }

    public LiveData<List<Item>> search(String query){
        return repository.searchItem(query);
    }

oraz metody zapisujące dane do bazy - tutaj wykorzystamy `ExecutorService` zdefiniowany w klasie `ItemDatabase`

In [ ]:
    public void update(Item item){
        ItemDatabase.databaseWriteExecutor
            .execute(() -> repository.update(item));
    }

    public void insert(Item item) {
        ItemDatabase.databaseWriteExecutor
            .execute(() -> repository.insert(item));
    }

    public void delete(Item item){
        ItemDatabase.databaseWriteExecutor
            .execute(() -> repository.delete(item));
    }

    public void deleteAll(){
        ItemDatabase.databaseWriteExecutor
            .execute(repository::deleteAll);
    }

### **nawigacja**

Do layoutu aktywności dodajemy `FragmentContainerView`

In [ ]:
<androidx.constraintlayout.widget.ConstraintLayout 
    xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".MainActivity">

    <androidx.fragment.app.FragmentContainerView
        android:id="@+id/fragmentContainerView"
        android:name="androidx.navigation.fragment.NavHostFragment"
        android:layout_width="409dp"
        android:layout_height="match_parent"
        app:defaultNavHost="true"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintHorizontal_bias="0.5"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent"
        app:navGraph="@navigation/navigation" />
</androidx.constraintlayout.widget.ConstraintLayout>

Layouty `AddFragment` oraz `UpdateFragment` będą zawierać dwa pola `EditText` oraz przycisk dzięki któremu wykonamy zapis/aktualizację

In [ ]:
<androidx.constraintlayout.widget.ConstraintLayout 
    xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".fragments.add.AddFragment">

    <EditText
        android:id="@+id/nameEditText"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:hint="@string/nazwa"
        android:textSize="24sp"
        android:gravity="center_horizontal"
        android:layout_marginStart="16dp"
        android:layout_marginTop="50dp"
        android:layout_marginEnd="16dp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent"
        android:inputType="text"
        android:importantForAutofill="no" />

    <EditText
        android:id="@+id/quantityEditText"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginStart="16dp"
        android:layout_marginTop="16dp"
        android:layout_marginEnd="16dp"
        android:gravity="center_horizontal"
        android:hint="@string/ilosc"
        android:importantForAutofill="no"
        android:textSize="24sp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/nameEditText"
        android:inputType="number" />

    <Button
        android:id="@+id/saveButton"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginStart="24dp"
        android:layout_marginTop="16dp"
        android:layout_marginEnd="24dp"
        android:text="@string/dodaj"
        android:textSize="24sp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/quantityEditText" />


</androidx.constraintlayout.widget.ConstraintLayout>

Layout `ListFragment` będzie zawiera `RecyclerView`, dwa `FloatingActionButton` (przejście do `AddFragment` oraz czyszczenie całej listy), oraz pole `SearchView`, które wykorzystamy do przeszukiwania bazy.

Dodajmy nawigację, do `ListFragment` dodajemy dwie akcje
- przejście do `AddFragment`
- przejście do `UpdateFragment` - tutaj przekazujemy `id`

In [ ]:
    <fragment
        android:id="@+id/listFragment"
        android:name="pl.udu.uwr.pum.shoppyjava.fragments.list.ListFragment"
        android:label="@string/lista"
        tools:layout="@layout/fragment_list" >
        <action
            android:id="@+id/action_listFragment_to_addFragment"
            app:destination="@id/addFragment" />
        <action
            android:id="@+id/action_listFragment_to_updateFragment"
            app:destination="@id/updateFragment" >
            <argument
                android:name="id"
                app:argType="integer" />
        </action>
    </fragment>

W pozostałych dwóch fragmentach definiujemy akcję przejścia powrotnego na `ListFragment`

In [ ]:
    <fragment
        android:id="@+id/addFragment"
        android:name="pl.udu.uwr.pum.shoppyjava.fragments.add.AddFragment"
        android:label="@string/dodaj"
        tools:layout="@layout/fragment_add" >
        <action
            android:id="@+id/action_addFragment_to_listFragment"
            app:destination="@id/listFragment" />
    </fragment>
    <fragment
        android:id="@+id/updateFragment"
        android:name="pl.udu.uwr.pum.shoppyjava.fragments.update.UpdateFragment"
        android:label="@string/edytuj"
        tools:layout="@layout/fragment_update" >
        <action
            android:id="@+id/action_updateFragment_to_listFragment"
            app:destination="@id/listFragment" />
    </fragment>

### **MainActivity**

W głównej aktywności umożliwimy nawigację wsteczną z poziomu `ActionBar`

In [ ]:
public class MainActivity extends AppCompatActivity {

    private NavController navController;
    private AppBarConfiguration appBarConfiguration;

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        ActivityMainBinding binding = 
            ActivityMainBinding.inflate(getLayoutInflater());
        setContentView(binding.getRoot());

        NavHostFragment navHostFragment = 
            (NavHostFragment) getSupportFragmentManager()
                .findFragmentById(R.id.fragmentContainerView);

        if (navHostFragment != null) {
            navController = 
                NavHostFragment.findNavController(navHostFragment);
        }

        appBarConfiguration =  
            new AppBarConfiguration.Builder(navController.getGraph())
                .build();

        NavigationUI.setupActionBarWithNavController(
            this, 
            navController, 
            appBarConfiguration);
    }

    @Override
    public boolean onSupportNavigateUp() {
        return NavigationUI
            .navigateUp(navController, appBarConfiguration) 
            || super.onSupportNavigateUp();
    }
}

### **AddFragment**

Rozpocznijmy od dodania `ItemViewModel`

In [ ]:
    private ItemViewModel viewModel;

Zainicjujemy go w metodzie `onCreate`

In [ ]:
    @Override
    public void onCreate(@Nullable Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);

        viewModel = new ViewModelProvider(this)
            .get(ItemViewModel.class);
    }

Zdefiniujmy metode `insertToDatabase` - przyjmuje ona jako argument `View`, który jest wymagany przy wywołaniu metody `findNavController`

In [ ]:
private void insertToDatabase(View view) {

W pierwszym kroku sprawdźmy czy pole `EditText` nie są puste

In [ ]:
    String name = binding.nameEditText.getText().toString();
    String quantity = binding.quantityEditText.getText().toString();

    if (TextUtils.isEmpty(name) && TextUtils.isEmpty(quantity)){

Jeżeli tak jest - wyświetlamy błąd

In [ ]:
    if (TextUtils.isEmpty(name) && TextUtils.isEmpty(quantity)){
        binding.nameEditText.setError("Podaj nazwę");
        binding.quantityEditText.setError("Podaj ilość");

W przeciwnym wypadku tworzymy nowy `Item` z dostępnych danych i wykonujemy metodę `insert` z klasy `ItemViewModel`

In [ ]:
    } else {
        Item item = new Item(0, name, Integer.parseInt(quantity));
        viewModel.insert(item);

Ostatnim elementem jest powrót do `ListFragemnt` po dodaniu nowego elementu

In [ ]:
    Navigation.findNavController(view)
        .navigate(AddFragmentDirections.actionAddFragmentToListFragment());

W metodzie `onViewCreated` dodajemy obsługę `onClick` przycisku odpowiadającego za aktualizację

In [ ]:
    binding.saveButton.setOnClickListener(v -> insertToDatabase(view));

### **UpdateFragment**

Rozpocznijmy od dodania pola `ItemViewModel` oraz `int` reprezentujący odebrane `id` z `ListFragment`

In [ ]:
private ItemViewModel itemViewModel;

private int itemId;

Inicjalizację wykonajmy w metodzie `onCreate`

In [ ]:
    @Override
    public void onCreate(@Nullable Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        itemViewModel = new ViewModelProvider(requireActivity()).get(ItemViewModel.class);
        itemId = requireArguments().getInt("id");
    }

Zdefiniujmy metodę wypełniającą pola `EditText` danymi pochodziącymi z elementu odpowiadającego `id` przekazanemu w arggumentach.

In [ ]:
private void displayItem(Item item) {
    binding.nameEditText.setText(item.getName());
    binding.quantityEditText
        .setText(String.valueOf(item.getQuantity()));
}

Dodajmy również metodę `updateItem`, która jest niemal identyczna jak poprzednio zdefiniowana `insertToDatabase`

In [ ]:
    private void updateItem(View view) {
        String name = binding.nameEditText.getText().toString();
        String quantity = binding.quantityEditText.getText().toString();

        if (TextUtils.isEmpty(name) && TextUtils.isEmpty(quantity)){
            binding.nameEditText.setError("Podaj nazwę");
            binding.quantityEditText.setError("Podaj ilość");
        } else {
            Item item = new Item(itemId, name, Integer.parseInt(quantity));
            itemViewModel.update(item);
            Navigation.findNavController(view)
                .navigate(UpdateFragmentDirections
                          .actionUpdateFragmentToListFragment());
        }
    }

W metodzie `onViewCreated` wyciągamy element z bazy za pomocą metody `getItem` i dodajemy obserwator - `displayItem`

In [ ]:
itemViewModel.getItem(itemId)
    .observe(getViewLifecycleOwner(), this::displayItem);

oraz zaimplementujmy metodę `onClick` przycisku aktualizującego element

In [ ]:
binding.updateButton.setOnClickListener(v -> updateItem(view));

### **ListFragment**

Rozpocznijmy implementację od zdefiniowania layoutu pojedynczego elementu

In [ ]:
<androidx.constraintlayout.widget.ConstraintLayout 
    xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:padding="24dp">

    <TextView
        android:id="@+id/nameTextViewRV"
        android:layout_width="0dp"
        android:layout_height="wrap_content"
        android:text="nazwa przedmiotu"
        android:textSize="24sp"
        android:textStyle="bold"
        app:layout_constraintEnd_toStartOf="@+id/guideline"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/quantityTextViewRV"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:gravity="end"
        android:text="999"
        android:textSize="24sp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="@+id/guideline"
        app:layout_constraintTop_toTopOf="parent" />

    <androidx.constraintlayout.widget.Guideline
        android:id="@+id/guideline"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:orientation="vertical"
        app:layout_constraintGuide_percent="0.75" />


</androidx.constraintlayout.widget.ConstraintLayout>

Dodajmy `ItemViewHolder`

In [ ]:
public class ItemViewHolder extends RecyclerView.ViewHolder {

    private final ItemRecyclerviewBinding binding;

    public ItemViewHolder(@NonNull ItemRecyclerviewBinding binding) {
        super(binding.getRoot());
        this.binding = binding;
    }

    public void bind(Item item){
        binding.nameTextViewRV.setText(item.getName());
        binding.quantityTextViewRV.setText(String.valueOf(item.getQuantity()));

        binding.getRoot().setOnClickListener(v -> {
            NavDirections action = ListFragmentDirections
                    .actionListFragmentToUpdateFragment(item.getId());
            Navigation.findNavController(binding.getRoot()).navigate(action);
        });
    }
}

Dodajemy implementację `onClick` elementu listy - przechodzimy do `UpdateFragment` przekazując `id` klikniętego elementu.

Dodajmy `ItemComparator` - dzięki niemu możemy zdefiniować metody porównujące elementy - jest wykorzystywany podczas wywołania `submitList` w klasie `Listfragment` (niejawnie)

In [ ]:
public class ItemComparator extends DiffUtil.ItemCallback<Item> {
    @Override
    public boolean areItemsTheSame(@NonNull Item oldItem, @NonNull Item newItem) {
        return oldItem == newItem;
    }

    @Override
    public boolean areContentsTheSame(@NonNull Item oldItem, @NonNull Item newItem) {
        return oldItem.getName().equals(newItem.getName()) &&
                oldItem.getQuantity() == newItem.getQuantity();
    }
}

Klasa `ItemAdapter` będzie rozszerzać `ListAdapter` - co ułatwi indeksowanie i zmianę danych

In [ ]:
public class ItemAdapter extends ListAdapter<Item, ItemViewHolder> {

    public ItemAdapter(ItemComparator comparator){
        super(comparator);
    }

    @NonNull
    @Override
    public ItemViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        return new ItemViewHolder(ItemRecyclerviewBinding.inflate(
                LayoutInflater.from(parent.getContext()), parent, false)
        );
    }

    @Override
    public void onBindViewHolder(@NonNull ItemViewHolder holder, int position) {
        Item item = getItem(position);
        holder.bind(item);
    }

    public Item getItemAt(int position){
        return getItem(position);
    }
}

Oprócz metod które musimy nadpisać, dodajemy metodę publiczną `getItemAt` zwracającą element na zadanej pozycji - wykorzystamy ją przy implementacji przeszukiwania w `ListFragment`.

W metodzie `onViewCreated` klasy `Listfragment` dodajmy `RecyclerView`

In [ ]:
ItemAdapter adapter = new ItemAdapter(new ItemComparator());
binding.listRecyclerView.setLayoutManager(new LinearLayoutManager(getContext()));
binding.listRecyclerView.setAdapter(adapter);

Następnie dodajmy obserwator

In [ ]:
itemViewModel.getAllData().observe(getViewLifecycleOwner(), adapter::submitList);

Dodajmy obsługę `onClick` dwóch `FAB` służących dodaniu i usunięciu wszystkich elementów

In [ ]:
binding.addItemFAB.setOnClickListener(v ->
        Navigation.findNavController(view)
                .navigate(ListFragmentDirections.actionListFragmentToAddFragment()));

binding.clearDataFAB.setOnClickListener(v -> itemViewModel.deleteAll());

#### **swipeToDelete**

zaimplementujmy funkcję `swipeToDelete` - będziemy usuwać elementu z bazy poprzez wykonanie zdarzenia `swipe` w lewo lub prawo. Jako argument metoda przyjmuje `ItemAdapter`. W pierwszym kroku tworzymy `ItemTouchHelper` - argumentem jest `SimpleCallback`. Jako `dragDirs` podajemy 0 (nie będziemy wykorzystywać tej funkcjonalności), a jako `swipeDirs` podajemy odpowiednie `int` zdefiniowane w klasie `ItemTouchHelper`

In [ ]:
    private void swipeToDelete(ItemAdapter adapter) {
        new ItemTouchHelper(new ItemTouchHelper.SimpleCallback(0,
                ItemTouchHelper.RIGHT | ItemTouchHelper.LEFT) {

Implementacja metody `onMove` nas nie interesuje, więc zwracamy `false`

In [ ]:
    @Override
    public boolean onMove(
        @NonNull RecyclerView recyclerView, 
        @NonNull RecyclerView.ViewHolder viewHolder, 
        @NonNull RecyclerView.ViewHolder target) {
        return false;
    }

W metodzie `onSwiped` wywołujemy `delete` klasy `ItemViewModel` - tutaj jako argument musimy podać element który chcemy usunąć, a mamy dostępną tylko pozycję elementu (przez metodę `getAdapterPosition`). W celu wyciągnięcia elementu wykorzystamy wcześniej zdefiniowaną metodę `getItemAt` w klasie `ItemAdapter`

In [ ]:
    @Override
    public void onSwiped(
        @NonNull RecyclerView.ViewHolder viewHolder, 
        int direction) {
        itemViewModel.delete(adapter.getItemAt(viewHolder.getAdapterPosition()));
    }
}).attachToRecyclerView(binding.listRecyclerView);

Metodę `swipeToDelete` wywołujemy w `onViewCreated`

In [ ]:
swipeToDelete(adapter);

#### **searchView**

Ostatnim elementem tej aplikacji jest implementacja `SearchView`, pozwalająca na przeszukanie bazy. W tym celu stwórzmy nową metodę `setupSearchView` - również tutaj jako parametr przekażemy `ItemAdapter`

In [ ]:
private void setupSearchView(ItemAdapter adapter) {

Tutaj dodamy `setOnQueryTextListener` do naszego `SearchView`, który wymaga implementacji dwóch metod
- `onQueryTextSubmit` - wykonana po zakończeniu edycji i zatwierdzeniu
- `onQueryTextChange` - wykonana przy każdej zmianie tekstu w polu `SearchView`

W obu tych metodach wykonamy tą samą metodę - `search`.

In [ ]:
    binding.searchSearchView.setOnQueryTextListener(
        new SearchView.OnQueryTextListener() {
        @Override
        public boolean onQueryTextSubmit(String query) {
            if (query != null) search(query, adapter);
            return true;
        }

        @Override
        public boolean onQueryTextChange(String newText) {
            if (newText != null) search(newText, adapter);
            return true;
        }
    });

Zdefiniujmy metodę `search` - argumentem jest tekst do wyszukania oraz `ItemAdapter`, w pierwszym kroku zdefiniujmy `String` reprezentujący frazę do wyszukania

In [ ]:
private void search(String query, ItemAdapter adapter){
    String searchQuery = "%" + query + "%";

Nastęnie na `ItemViewModel` wykonajmy metodę `search` zwracającą listę wszystkich elementów pasujących do frazy wyszukania, oraz dodajmy obserwator i wykonajmy `submitList` - zaowocuje to natychmiastowym wyświetleniem wyniku w `RecyclerView`.

In [ ]:
    itemViewModel.search(searchQuery)
        .observe(getViewLifecycleOwner(), adapter::submitList);

Możemy przetestować aplikację